# Before Analysis...

In [ ]:
import numpy as np
import mne
import sklearn
import matplotlib.pyplot as plt
from scipy import stats
from decimal import Decimal, getcontext
import pandas as pd
import ast
import openpyxl as op
import re

In [ ]:
# 분석대상 이름 입력
# 실행 전 경로 및 파일명 확인 필요

name_raw = input("분석대상 이름을 입력하세요.")

# Loading saved TFR file
file_path = rf'H:\Mg_EEG\tfr_files_gamma\{name_raw}_7200_tfr.h5'
tfr = mne.time_frequency.read_tfrs(file_path)
print(f"성공적으로 {name_raw}의 tfr 파일을 로딩하였습니다.")

# Saving raw file for back-up
tfr_raw = tfr.copy()
print(f"{name_raw}의 raw 파일을 백업하였습니다.")

# plot 작성을 위한 TFR 평균 계산산
def tfr_mean(tfr):
    mean_electrode = np.mean(tfr.data, axis=0) # Averaging across electrode
    mean_frequency = np.mean(mean_electrode, axis=0) # Averaging across frequency
    return(mean_frequency)

# name에서 알파벳만 제거하여 이름 복구
name = re.sub(r'[a-zA-Z]', '', name_raw)
print(name)
print(f"{name}의 TFR을 분석합니다.")

In [ ]:
#raw fif file 보고 싶다면 사용
# clean_data.plot()

In [ ]:
#데이터가 뭔가 이상할 때 리셋하는 복구 코드
#tfr = tfr_raw

In [ ]:
# large artifact data load
csv_file_path = r'C:\Users\esin4\OneDrive\바탕 화면\Github\Mg_infusion_coma\large_artifact.csv'

large_artifact_data = pd.read_csv(csv_file_path, encoding='utf-8-sig')

# 입력한 이름이 데이터프레임에 존재하는지 확인하고 좌표 가져오기
if name in large_artifact_data['Name'].values:
    # 입력한 이름에 해당하는 데이터 가져오기
    coordinates = large_artifact_data[large_artifact_data['Name'] == name]['Coordinates'].values[0]
    
    # 좌표가 'skip'이면 빈 리스트로 설정
    if coordinates == 'skip':
        large_artifact = []
    else:
        # 좌표 문자열을 리스트로 변환
        import ast
        large_artifact = ast.literal_eval(coordinates)
# 입력한 이름이 데이터프레임에 없으면 빈 리스트로 설정
else:
    print(f"{name}이(가) 데이터에 없습니다.")
    large_artifact = []

# 결과 출력
print("large_artifact 리스트:")
print(large_artifact)

In [6]:
# 분석한 값 저장할 엑셀 파일
wb = op.load_workbook(r"C:\Users\esin4\OneDrive\바탕 화면\Github\Mg_infusion_coma\Mg_infusion_data.xlsx")
row = int(input("데이터가 입력될 행을 입력하세요."))

# independent t-test

In [ ]:
# Before vs After segmentation
tfr_before = tfr.copy().crop(tmin=0, tmax=3600, include_tmax=False)
tfr_after = tfr.copy().crop(tmin=3600, tmax=7199, include_tmax=False)

# Averaging
before_mean = tfr_mean(tfr_before)
after_mean = tfr_mean(tfr_after)

# Indexing about Large artifact
before_idx = []
after_idx = []

for t_start, t_end in large_artifact:
    if t_start < 3600 and t_end < 3600:
        before_idx.extend(range(t_start*200, t_end*200))
    elif t_start < 3600 and t_end >= 3600:
        before_idx.extend(range(t_start*200, 3600*200))
        after_idx.extend(range(0, (t_end-3600)*200))
    else:
        after_idx.extend(range((t_start-3600)*200, (t_end-3600)*200))

# Removing Large artifacts
before_mean_clean = np.delete(before_mean, before_idx)
after_mean_clean = np.delete(after_mean, after_idx)

# independent t-test
# precision
getcontext().prec = 50

print(f"{name}, independent t test")
# mean of before vs after
mean_before = np.mean(before_mean_clean)
mean_after = np.mean(after_mean_clean)
print("mean_before : \n", mean_before)
print("mean_after : \n", mean_after)

# t-test
t_stat, p_value = stats.ttest_ind(before_mean_clean, after_mean_clean)

# print to decimal object
p_value_decimal = Decimal(p_value)
print ("t_statistics : \n", t_stat)
p_value_str = f"{p_value_decimal:.100f}"
p_value_num = float(p_value_str)
print(f"p-value: \n {p_value_str}")

# Define Cohens'D 
def cohens_d(group1, group2):
    # Calculate mean, sd
    mean1, mean2 = np.mean(group1), np.mean(group2)
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    
    # sample size
    n1, n2 = len(group1), len(group2)
    
    # Calculate pooled sd
    pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))
    
    # Cohen's d
    d = (mean1 - mean2) / pooled_std
    return d

# Calculate Effect Size
effect_size = cohens_d(before_mean, after_mean)
print("effect size \n", effect_size)

In [ ]:
# Save to Excel
ws_i = wb["independent t"]
ws_i.cell(row=row, column=1).value = f"{name}"
ws_i.cell(row=row, column=2).value = mean_before
ws_i.cell(row=row, column=3).value = mean_after
ws_i.cell(row=row, column=4).value = t_stat
ws_i.cell(row=row, column=5).value = p_value
ws_i.cell(row=row, column=7).value = effect_size

# Paired t-test

In [ ]:
# Before vs After segmentation
tfr_before = tfr.copy().crop(tmin=0, tmax=3600, include_tmax=False)
tfr_after = tfr.copy().crop(tmin=3600, tmax=7199, include_tmax=False)

# Averaging
before_mean = tfr_mean(tfr_before)
after_mean = tfr_mean(tfr_after)

# trim the data 무조건 7200초 맞춰서 이 부분 안 써도 되도록 해야함. 
if len(before_mean) > len(after_mean):
    before_mean = before_mean[:len(after_mean)]
elif len(before_mean) < len(after_mean):
    after_mean = after_mean[:len(before_mean)]
else:
    quit

len(before_mean) == len(after_mean)

# Indexing about large artifact

before_idx = set()
after_idx = set()

for t_start, t_end in large_artifact:
    if t_start < 3600 and t_end < 3600:
        before_idx.update(range(t_start*200, t_end*200))
        after_idx.update(range(t_start*200, t_end*200))
    elif t_start < 3600 and t_end >= 3600:
        before_idx.update(range(t_start*200, 3600*200))
        after_idx.update(range(t_start*200, 3600*200))
        after_idx.update(range(0, (t_end-3600)*200))
        before_idx.update(range(0, (t_end-3600)*200))
    else:
        after_idx.update(range((t_start-3600)*200, (t_end-3600)*200))
        before_idx.update(range((t_start-3600)*200, (t_end-3600)*200))

# Convert sets back to sorted lists
before_idx = sorted(before_idx)
after_idx = sorted(after_idx)

before_mean_clean = np.delete(before_mean, before_idx)
after_mean_clean = np.delete(after_mean, after_idx)

# Paired t-test
# precision
getcontext().prec = 50

print(f"{name}, paired t test")
# mean of before vs after
mean_before = np.mean(before_mean_clean)
mean_after = np.mean(after_mean_clean)
print("mean_before :\n ", mean_before)
print("mean_after : \n", mean_after)

# Kolmogorov-Smirnov test
difference = before_mean_clean-after_mean_clean
ks_statistic, ks_p_value = stats.kstest(difference, stats.norm.cdf)
print(f"KS Statistic: \n {ks_statistic}")
print(f"KS p-value: \n {ks_p_value}")

# t-test
t_stat, p_value = stats.ttest_rel(before_mean, after_mean)

# print to decimal object
p_value_decimal = Decimal(p_value)
print ("t_statistics : \n", t_stat)
p_value_str = f"{p_value_decimal:.100f}"
print(f"p-value: \n {p_value_str}")

# Define Cohens'D 
def cohens_d(group1, group2):
    # Calculate mean, sd
    mean1, mean2 = np.mean(group1), np.mean(group2)
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    
    # sample size
    n1, n2 = len(group1), len(group2)
    
    # Calculate pooled sd
    pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))
    
    # Cohen's d
    d = (mean1 - mean2) / pooled_std
    return d

# Calculate Effect Size
effect_size = cohens_d(before_mean, after_mean)
print("effect size \n", effect_size)

In [ ]:
# Save to Excel
ws_p = wb["paired t"]
ws_p.cell(row=row, column=1).value = f"{name}"
ws_p.cell(row=row, column=2).value = mean_before
ws_p.cell(row=row, column=3).value = mean_after
ws_p.cell(row=row, column=4).value = ks_statistic
ws_p.cell(row=row, column=5).value = ks_p_value
ws_p.cell(row=row, column=7).value = t_stat
ws_p.cell(row=row, column=8).value = p_value
ws_p.cell(row=row, column=10).value = effect_size

# Analysis across Band

## Paired t-test with Cohen's D test

In [ ]:
# 주파수 대역별로 처리
freq_ranges = {
    'Delta': (1, 4),
    'Theta': (4, 8),
    'Alpha': (8, 13),
    'Beta': (13, 30),
    'gamma': (30, 90)
}

# Averaging by band and timing
for band, (fmin, fmax) in freq_ranges.items():
    tfr_band_before = tfr.copy().crop(tmin=0, tmax=3600, fmin=fmin, fmax=fmax, include_tmax=False)
    tfr_band_after = tfr.copy().crop(tmin=3600, tmax=7199, fmin=fmin, fmax=fmax, include_tmax=False)

    globals()[f'tfr_{band}_before'] = tfr_band_before
    globals()[f'tfr_{band}_after'] = tfr_band_after

    globals()[f'{band}_before_mean'] = tfr_mean(tfr_band_before)
    globals()[f'{band}_after_mean'] = tfr_mean(tfr_band_after)

# Make length same    
for band in freq_ranges.keys():
    before_mean = globals()[f'{band}_before_mean']
    after_mean = globals()[f'{band}_after_mean']

    if len(before_mean) > len(after_mean):
        globals()[f'{band}_before_mean'] = before_mean[:len(after_mean)]
    elif len(before_mean) < len(after_mean):
        globals()[f'{band}_after_mean'] = after_mean[:len(before_mean)]
    else:
        print(f"{band} 대역의 before와 after 길이가 이미 같습니다.")

    print(f"{band} 대역 처리 완료: before 길이 = {len(globals()[f'{band}_before_mean'])}, after 길이 = {len(globals()[f'{band}_after_mean'])}")

# Removing large artifact by band
def remove_artifacts(before_mean, after_mean, large_artifact):
    before_idx = set()
    after_idx = set()

    for t_start, t_end in large_artifact:
        if t_start < 3600 and t_end < 3600:
            before_idx.update(range(t_start*200, t_end*200))
            after_idx.update(range(t_start*200, t_end*200))
        elif t_start < 3600 and t_end >= 3600:
            before_idx.update(range(t_start*200, 3600*200))
            after_idx.update(range(t_start*200, 3600*200))
            after_idx.update(range(0, (t_end-3600)*200))
            before_idx.update(range(0, (t_end-3600)*200))
        else:
            after_idx.update(range((t_start-3600)*200, (t_end-3600)*200))
            before_idx.update(range((t_start-3600)*200, (t_end-3600)*200))

    before_idx = sorted(before_idx)
    after_idx = sorted(after_idx)

    before_mean_clean = np.delete(before_mean, before_idx)
    after_mean_clean = np.delete(after_mean, after_idx)

    return before_mean_clean, after_mean_clean

# 주파수 대역별로 artifact 제거
for band in freq_ranges.keys():
    before_mean = globals()[f'{band}_before_mean']
    after_mean = globals()[f'{band}_after_mean']

    before_mean_clean, after_mean_clean = remove_artifacts(before_mean, after_mean, large_artifact)

    globals()[f'{band}_before_mean_clean'] = before_mean_clean
    globals()[f'{band}_after_mean_clean'] = after_mean_clean

    print(f"{band} 대역 처리 완료:")
    print(f"  Before: 원본 길이 = {len(before_mean)}, 정제 후 길이 = {len(before_mean_clean)}")
    print(f"  After: 원본 길이 = {len(after_mean)}, 정제 후 길이 = {len(after_mean_clean)}")
    print(f"  정제 후 Before와 After 길이 일치: {len(before_mean_clean) == len(after_mean_clean)}")

print(f"\n {name}, paired t test by band")

# 테스트 실행 함수
def run_tests(before_mean, after_mean, band_name):
    print(f"\n=== {band_name} 대역 분석 결과 ===")
    
    # precision
    getcontext().prec = 50

    # mean of before vs after
    mean_before = np.mean(before_mean)
    mean_after = np.mean(after_mean)
    print(f"mean_before: \n {mean_before}")
    print(f"mean_after: \n {mean_after}")

    # Kolmogorov-Smirnov test
    difference = before_mean - after_mean
    ks_statistic, ks_p_value = stats.kstest(difference, stats.norm.cdf)
    print(f"KS Statistic: \n {ks_statistic}")
    print(f"KS p-value: \n {ks_p_value}")

    # t-test
    t_stat, p_value = stats.ttest_rel(before_mean, after_mean)

    # print to decimal object
    p_value_decimal = Decimal(p_value)
    print(f"t_statistics: \n {t_stat}")
    p_value_str = f"{p_value_decimal:.100f}"
    print(f"p-value: \n {p_value_str}")

    # Define Cohens'D 
    def cohens_d(group1, group2):
        mean1, mean2 = np.mean(group1), np.mean(group2)
        std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
        n1, n2 = len(group1), len(group2)
        pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))
        d = (mean1 - mean2) / pooled_std
        return d

    # Calculate Effect Size
    effect_size = cohens_d(before_mean, after_mean)
    print(f"Effect size: \n {effect_size}")

    # 엑셀에 저장
    ws_b = wb["Sheet1"]
    if band_name == 'Delta':
        row_b = row*5-8
    elif band_name == 'Theta':
        row_b = row*5-7
    elif band_name == 'Alpha':
        row_b = row*5-6
    elif band_name == 'Beta':
        row_b = row*5-5
    elif band_name == 'gamma':
        row_b = row*5-4
    else:
        print("오류가 발생하였습니다.")
        return
    
    ws_b.cell(row=row_b, column=1).value = f"{name}"
    ws_b.cell(row=row_b, column=4).value = mean_before
    ws_b.cell(row=row_b, column=5).value = mean_after
    ws_b.cell(row=row_b, column=6).value = ks_statistic
    ws_b.cell(row=row_b, column=7).value = ks_p_value
    ws_b.cell(row=row_b, column=9).value = t_stat
    ws_b.cell(row=row_b, column=10).value = p_value
    ws_b.cell(row=row_b, column=12).value = effect_size

# 각 주파수 대역별로 테스트 실행
for band in freq_ranges.keys():
    before_mean = globals()[f'{band}_before_mean_clean']
    after_mean = globals()[f'{band}_after_mean_clean']
    run_tests(before_mean, after_mean, band)

In [8]:
# Save DATA to Excel
wb.save(r"C:\Users\esin4\OneDrive\바탕 화면\Github\Mg_infusion_coma\Mg_infusion_data.xlsx")

## 실험적 접근

Relative Power

In [ ]:
import pandas as pd

# Load the specific sheet into a DataFrame
sheet_name = input("불러올 시트 이름을 입력하세요: ")
df = pd.read_excel(r"C:\Users\esin4\OneDrive\바탕 화면\Github\Mg_infusion_coma\Mg_infusion_data.xlsx", sheet_name=sheet_name)

# Display the DataFrame
display(df)

In [3]:
df_pre = df.loc[:, ['name', 'band', 'mean_before', 'mean_after']]

In [4]:
df_pre['Effect'] = df_pre['mean_before'] > df_pre['mean_after']

In [ ]:
# 전체 중 효과있었던 밴드의 비율
(df_pre.groupby('band')['Effect'].sum())/len(df_pre)

In [6]:
df_pivot = df_pre.pivot(index='name', columns='band', values='Effect').reset_index()

In [7]:
df_pivot = df_pivot[['name', 'delta', 'theta', 'alpha', 'beta', 'gamma']]

In [ ]:
df_pivot

In [9]:
eye_list = ['Y', 'Y-N', 'N', 'N', 'Y', 'Y-N', 'N', 'N', 'N', 'N', 'N', 'N', 'Y', 'N', 'N', 'Y', 'N', 'N', 'N', 'N', 'N', 'N']

In [10]:
df_pivot['Eye'] = eye_list

In [ ]:
df_pivot

In [12]:
df_pivot['Eye_TF'] = (df_pivot['Eye'] == 'Y') | (df_pivot['Eye'] == 'Y-N')

In [ ]:
df_pivot.loc[df_pivot['Eye_TF']==True]

In [ ]:
import scipy.stats as stats
from scipy.stats import chi2_contingency
import numpy as np

def analyze_categorical_association(df, wave_band):
    # 교차표 생성
    contingency = pd.crosstab(df[wave_band], df['Eye_TF'])
    
    # 카이제곱 검정
    chi2, p_value, dof, expected = chi2_contingency(contingency)
    
    # Cramer's V 계산
    n = contingency.sum().sum()
    min_dim = min(contingency.shape) - 1
    cramer_v = np.sqrt(chi2 / (n * min_dim))
    
    return {
        'wave_band': wave_band,
        'chi2': chi2,
        'p_value': p_value,
        'cramer_v': cramer_v
    }

# 각 뇌파 대역에 대해 분석 수행
results = []
for band in ['delta', 'theta', 'alpha', 'beta', 'gamma']:
    result = analyze_categorical_association(df_pivot, band)
    results.append(result)

# 결과를 데이터프레임으로 정리
results_df = pd.DataFrame(results)
print("\n연관성 분석 결과:")
print(results_df.round(4))

In [15]:
# relative power 계산
df_pre['mean_before_total'] = df_pre.groupby('name')['mean_before'].transform('sum')
df_pre['mean_after_total'] = df_pre.groupby('name')['mean_after'].transform('sum')
df_pre['relative_before'] = df_pre['mean_before'] / df_pre['mean_before_total']
df_pre['relative_after'] = df_pre['mean_after'] / df_pre['mean_after_total']

In [ ]:
df_pre

In [ ]:
eye_list1 = []
for i in eye_list:
    eye_list1.append(i)
    eye_list1.append(i)
    eye_list1.append(i)
    eye_list1.append(i)
    eye_list1.append(i)
eye_list1

In [17]:
df_pre['Eye_TF'] = eye_list1

In [ ]:
df_pre

In [ ]:
def prepare_data(data_df):
    """
    원본 데이터를 Repeated Measures ANOVA에 적합한 형태로 변환
    
    Parameters:
    data_df: DataFrame with columns ['name', 'band', 'relative_before', 'relative_after']
    
    Returns:
    DataFrame with columns ['name', 'band', 'Condition', 'mean']
    """
    # 데이터를 long format으로 변환
    before_df = data_df[['name', 'band', 'mean_before', 'relative_before']].rename(
        columns={'relative_before': 'relative_power', 'mean_before': 'mean_power'})
    before_df['Condition'] = 'before'
    
    after_df = data_df[['name', 'band', 'mean_after', 'relative_after']].rename(
        columns={'relative_after': 'relative_power', 'mean_after': 'mean_power'})
    after_df['Condition'] = 'after'
    
    # 데이터 합치기
    long_df = pd.concat([before_df, after_df], ignore_index=True)
    
    return long_df

prepare_data(df_pre)

In [40]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.anova import AnovaRM
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def prepare_data(data_df):
    """
    원본 데이터를 Repeated Measures ANOVA에 적합한 형태로 변환
    
    Parameters:
    data_df: DataFrame with columns ['name', 'band', 'relative_before', 'relative_after']
    
    Returns:
    DataFrame with columns ['name', 'band', 'Condition', 'mean']
    """
    # 데이터를 long format으로 변환
    before_df = data_df[['name', 'band', 'relative_before']].rename(
        columns={'relative_before': 'relative_power'})
    before_df['Condition'] = 'before'
    
    after_df = data_df[['name', 'band', 'relative_after']].rename(
        columns={'relative_after': 'relative_power'})
    after_df['Condition'] = 'after'
    
    # 데이터 합치기
    long_df = pd.concat([before_df, after_df], ignore_index=True)
    
    return long_df

def analyze_eeg_data(data_df):
    """
    Two-way Repeated Measures ANOVA 분석 함수
    
    Parameters:
    data_df: DataFrame with columns ['name', 'band', 'relative_before', 'relative_after']
    """
    # 데이터 변환
    long_data = prepare_data(data_df)
    
    # 1. Repeated Measures ANOVA 실행
    rm_anova = AnovaRM(long_data, 'relative_power', 'name', within=['Condition', 'band'])
    result = rm_anova.fit()
    
    # 2. 정규성 검정
    conditions = ['before', 'after']
    bands = data_df['band'].unique()
    
    print("Shapiro-Wilk Normality Test:")
    for cond in conditions:
        for band in bands:
            subset = long_data[(long_data['Condition'] == cond) & 
                             (long_data['band'] == band)]['relative_power']
            stat, p = stats.shapiro(subset)
            print(f"{cond}-{band}: W={stat:.3f}, p={p:.3f}")
    
    # 3-1. 사후 분석 (by T test)
    def post_hoc_analysis_ttest(data_df):
        post_hoc_results = []
        for band in bands:
            band_data = data_df[data_df['band'] == band]
            t_stat, p_val = stats.ttest_rel(
                band_data['relative_before'],
                band_data['relative_after']
            )
            post_hoc_results.append({
                'band': band,
                't_statistic': t_stat,
                'p_value': p_val
            })
        return pd.DataFrame(post_hoc_results)
    

    # 3-2. 사후 분석 (by Tukey HSD)
    def post_hoc_analysis_tukey(data_df):
        """
        Tukey HSD를 사용한 post-hoc 분석
    
        Parameters:
        data_df: DataFrame with columns ['name', 'band', 'relative_before', 'relative_after']
    
        Returns:
        Dictionary containing Tukey HSD results for each band
        """
        # 데이터를 long format으로 변환
        long_data = prepare_data(data_df)
    
        tukey_results = {}
        for band in data_df['band'].unique():
            # 해당 밴드의 데이터만 선택
            band_data = long_data[long_data['band'] == band]
        
            # Tukey HSD 분석 수행
            tukey = pairwise_tukeyhsd(
                endog=band_data['relative_power'],
                groups=band_data['Condition'],
                alpha=0.05
            )
        
            tukey_results[band] = tukey
    
        return tukey_results
    
    # 4. 시각화
    def plot_results(long_data):
        # Boxplot
        plt.figure(figsize=(12, 6))
        sns.boxplot(x='band', y='relative_power', hue='Condition', data=long_data)
        plt.title('Mean Values by Band and Condition')
        plt.show()
        
        # Interaction plot
        plt.figure(figsize=(12, 6))
        means = long_data.groupby(['band', 'Condition'])['relative_power'].mean().unstack()
        means.plot(marker='o')
        plt.title('Interaction Plot: Band x Condition')
        plt.ylabel('Relative Power Value')
        plt.grid(True)
        plt.show()
    
    return {
        'anova_results': result,
        'post_hoc_ttest': post_hoc_analysis_ttest(data_df),
        'post_hoc_tukey': post_hoc_analysis_tukey(data_df),
        'plot_function': lambda: plot_results(long_data)
    }

In [ ]:
results = analyze_eeg_data(df_pre)

In [ ]:
print(results['anova_results'])

In [ ]:
print(results['post_hoc_ttest'])

In [ ]:
for band, tukey_result in results['post_hoc_tukey'].items():
    print(f"Results for {band} band:")
    print(tukey_result.summary())
    print()


In [ ]:
# 결과 확인
print(results['anova_results'])
print(results['post_hoc_ttest'])
results['plot_function']()

In [ ]:
df_pivot

In [ ]:
import itertools

# 밴드 리스트
bands = ['delta', 'theta', 'alpha', 'beta', 'gamma']

data = df_pivot.copy()

# 결과 저장
results = []

# 모든 조합 생성
for r in range(1, len(bands) + 1):  # 조합 크기 1부터 모든 밴드까지
    for combination in itertools.combinations(bands, r):
        # 진단 기준: 조합에 포함된 모든 밴드가 True인 경우
        data['diagnosis'] = data[list(combination)].all(axis=1)
        
        # 혼동 행렬 요소 계산
        TP = ((data['diagnosis'] == True) & (data['Eye_TF'] == True)).sum()
        TN = ((data['diagnosis'] == False) & (data['Eye_TF'] == False)).sum()
        FP = ((data['diagnosis'] == True) & (data['Eye_TF'] == False)).sum()
        FN = ((data['diagnosis'] == False) & (data['Eye_TF'] == True)).sum()
        
        # 민감도와 특이도 계산
        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        
        # 결과 저장
        results.append({
            'combination': combination,
            'sensitivity': sensitivity,
            'specificity': specificity,
            'TP': TP, 'TN': TN, 'FP': FP, 'FN': FN
        })

# 결과를 데이터프레임으로 정리
results_df = pd.DataFrame(results)

# 결과 출력
display(results_df)